In [59]:
import pandas as pd
import numpy as np
import json
import os
from striprtf.striprtf import rtf_to_text
import re
from tqdm import tqdm
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn import svm 
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

import random
random.seed(42)

# Loading data

In [3]:
df = pd.read_csv('dataset_fixed_v3/train.csv')
df_test = pd.read_csv('dataset_fixed_v3/test.csv')
df = pd.concat([df, df_test])
df = df[df.theme == 1][['text', 'kmonitor_tags']]
df

,text,kmonitor_tags
2,A 8. rendű vádlott egy szerelmi kaland miatt b...,"Zuschlag-ügy,csalás,magánokirat-hamisítás"
3,A Rendvédelmi Szervek Védelmi Szolgálata (RSZV...,"Olajügyek,Olajügyek,bűnpártolás,Békés megye"
4,A B Build & Trade Kft. építheti fel az új spor...,"közbeszerzés,sport,Baranya megye,Pécs"
9,"Az MVM-nél belső vizsgálat zajlik, mert több m...","MVM-ügyek 2005-2008,állami / önkormányzati vál..."
11,Gyakorlatilag visszavonta feltáró jellegű nyom...,"BKV-ügyek,állami / önkormányzati vállalat,hűtl..."
...,...,...
3611,"A Fideszben többen is biztosra veszik, hogy Sc...","plágium,sport,tudomány"
3612,A magyar kormány kezdeményezni fogja a norvég ...,"Kormány vs. Norvég Alap,átláthatóság,belföld,c..."
3614,Az FATF (Financial Action Task Force) elnöke C...,"belföld,pénzmosás"
3616,Adócsalás? Megkezdődött Olmert pere\nIzraelben...,Izrael


In [29]:
f = open('tags.json')
tags = json.load(f)['tags']

In [7]:
df.kmonitor_tags.iloc[0]

'Zuschlag-ügy,csalás,magánokirat-hamisítás'

In [31]:
def get_tag_columns(df: pd.DataFrame) -> tuple[pd.DataFrame, list[str]]:
    df = df.copy()
    tag_columns = []
    for tag in tqdm(tags):
        one_hot = df.apply(lambda x: 1 if str(tag) in str(x['kmonitor_tags']) else 0, axis = 1)
        if one_hot.sum() > 0:
            df["tag_" + tag] = df.apply(lambda x: 1 if str(tag) in str(x['kmonitor_tags']) else 0, axis = 1)
            tag_columns = tag_columns + ["tag_" + tag]
    return df, tag_columns
df, tag_columns = get_tag_columns(df)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 252/252 [01:17<00:00,  3.24it/s]


In [32]:
df

,text,kmonitor_tags,tag_ingatlan,tag_klientúra,tag_közbeszerzés,tag_önkormányzat,tag_támogatás,tag_átláthatóság,tag_háttér,tag_gazdálkodás,...,tag_Király utca,tag_műjégpálya,tag_traffipax,tag_szabályozott ingatlanbefektetési társaság (szit),tag_NVSZ-vizsgálat,tag_igazoltatás,tag_megbízhatósági vizsgálat,tag_közigazgatási bíróság,tag_kárpótlási jegy,tag_közjegyző
2,A 8. rendű vádlott egy szerelmi kaland miatt b...,"Zuschlag-ügy,csalás,magánokirat-hamisítás",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,A Rendvédelmi Szervek Védelmi Szolgálata (RSZV...,"Olajügyek,Olajügyek,bűnpártolás,Békés megye",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,A B Build & Trade Kft. építheti fel az új spor...,"közbeszerzés,sport,Baranya megye,Pécs",0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,"Az MVM-nél belső vizsgálat zajlik, mert több m...","MVM-ügyek 2005-2008,állami / önkormányzati vál...",0,0,0,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
11,Gyakorlatilag visszavonta feltáró jellegű nyom...,"BKV-ügyek,állami / önkormányzati vállalat,hűtl...",0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3611,"A Fideszben többen is biztosra veszik, hogy Sc...","plágium,sport,tudomány",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3612,A magyar kormány kezdeményezni fogja a norvég ...,"Kormány vs. Norvég Alap,átláthatóság,belföld,c...",0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3614,Az FATF (Financial Action Task Force) elnöke C...,"belföld,pénzmosás",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3616,Adócsalás? Megkezdődött Olmert pere\nIzraelben...,Izrael,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
tag_stats = df[tag_columns].sum(axis = 0).sort_values(ascending=False).to_frame()
tag_stats.index.name = 'tag'
tag_stats.columns = ['n']
tag_stats['ratio%'] = round((tag_stats['n'] / df.shape[0]) * 100, 2)
tag_stats

,n,ratio%
tag,,
tag_önkormányzat,6188,21.80
tag_klientúra,3960,13.95
tag_közbeszerzés,3515,12.39
tag_ingatlan,3220,11.35
tag_támogatás,3161,11.14
...,...,...
tag_közigazgatási bíróság,4,0.01
tag_közfeladati helyzettel visszaélés,3,0.01
tag_traffipax,3,0.01


In [34]:
tag_stats.describe()

,n,ratio%
count,247.000000,247.000000
mean,396.263158,1.395992
std,756.489383,2.665523
min,3.000000,0.010000
25%,36.000000,0.130000
50%,116.000000,0.410000
75%,378.000000,1.335000
max,6188.000000,21.800000


In [35]:
df.text

2       A 8. rendű vádlott egy szerelmi kaland miatt b...
3       A Rendvédelmi Szervek Védelmi Szolgálata (RSZV...
4       A B Build & Trade Kft. építheti fel az új spor...
9       Az MVM-nél belső vizsgálat zajlik, mert több m...
11      Gyakorlatilag visszavonta feltáró jellegű nyom...
                              ...                        
3611    A Fideszben többen is biztosra veszik, hogy Sc...
3612    A magyar kormány kezdeményezni fogja a norvég ...
3614    Az FATF (Financial Action Task Force) elnöke C...
3616    Adócsalás? Megkezdődött Olmert pere\nIzraelben...
3617    A BKV Zrt. és a vállalat pályázatain nyertes c...
Name: text, Length: 28381, dtype: object

# Cleaning and coding data

In [39]:
def get_stop_words()->list[str]:
    f = open('stopwords-hu.json')
    stop_words = json.load(f) +  ['hétfő', 'kedd', 'szerda', 'csütörtök', 'péntek', 'szombat', 'vasárnap', 'hétköznap', 'hétvége', 'év', 'hónap', 'egy']
    stop_words = stop_words + ['ide', 'lány', 'fiú', 'napos', 'tavasz', 'nyár', 'ősz', 'tél', 'amennyi', 'eddig', 'addig', 'idő', 'eső', 'lépés', 'amelyből', 'ahhoz', 'gondolom', 'gondolja', 'hasonló', 'megmondani', 'mondom', 'olyan', 'ilyen', 'ottan', 'soha', 'sose', 'sohase']
    f.close()
    return stop_words

def clean_text(text: str) -> str:
    stop_words = get_stop_words()
    
    html = re.compile(r'<.*?>')
    url = re.compile(r'http\S+')
    
    text = text.lower()
    text = url.sub('',text)
    text = html.sub('',text)

    punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`" + '_' + '0123456789' + '"'
    for p in punctuations:
        text = text.replace(p,' ')

    text = [word.lower().strip() for word in text.split()]
    text = [word for word in text if word not in stop_words]
    return " ".join(text)

In [40]:
df['clean_text'] = [clean_text(text) for text in df.text]
df.head(3)

,text,kmonitor_tags,tag_ingatlan,tag_klientúra,tag_közbeszerzés,tag_önkormányzat,tag_támogatás,tag_átláthatóság,tag_háttér,tag_gazdálkodás,...,tag_műjégpálya,tag_traffipax,tag_szabályozott ingatlanbefektetési társaság (szit),tag_NVSZ-vizsgálat,tag_igazoltatás,tag_megbízhatósági vizsgálat,tag_közigazgatási bíróság,tag_kárpótlási jegy,tag_közjegyző,clean_text
2,A 8. rendű vádlott egy szerelmi kaland miatt b...,"Zuschlag-ügy,csalás,magánokirat-hamisítás",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,rendű vádlott szerelmi kaland bonyolódott zusc...
3,A Rendvédelmi Szervek Védelmi Szolgálata (RSZV...,"Olajügyek,Olajügyek,bűnpártolás,Békés megye",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,rendvédelmi szervek védelmi szolgálata rszvsz ...
4,A B Build & Trade Kft. építheti fel az új spor...,"közbeszerzés,sport,Baranya megye,Pécs",0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,b build trade kft építheti sportcsarnokot foci...


In [41]:
def stem_text(text: str) -> str:
    stemmer = SnowballStemmer("hungarian")
    return ' '.join([stemmer.stem(word) for word in text.split(' ')])

In [42]:
df['clean_stemmed_text'] = [stem_text(text) for text in df['clean_text']]
df.head(3)

,text,kmonitor_tags,tag_ingatlan,tag_klientúra,tag_közbeszerzés,tag_önkormányzat,tag_támogatás,tag_átláthatóság,tag_háttér,tag_gazdálkodás,...,tag_traffipax,tag_szabályozott ingatlanbefektetési társaság (szit),tag_NVSZ-vizsgálat,tag_igazoltatás,tag_megbízhatósági vizsgálat,tag_közigazgatási bíróság,tag_kárpótlási jegy,tag_közjegyző,clean_text,clean_stemmed_text
2,A 8. rendű vádlott egy szerelmi kaland miatt b...,"Zuschlag-ügy,csalás,magánokirat-hamisítás",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,rendű vádlott szerelmi kaland bonyolódott zusc...,rendű vádlot szerelm kalan bonyolódot zuschlag...
3,A Rendvédelmi Szervek Védelmi Szolgálata (RSZV...,"Olajügyek,Olajügyek,bűnpártolás,Békés megye",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,rendvédelmi szervek védelmi szolgálata rszvsz ...,"rendvédelm szerv védelm szolgálat rszvsz , töb..."
4,A B Build & Trade Kft. építheti fel az új spor...,"közbeszerzés,sport,Baranya megye,Pécs",0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,b build trade kft építheti sportcsarnokot foci...,b buil trad kft építhet sportcsarn focipály pé...


In [43]:
df.clean_text

2       rendű vádlott szerelmi kaland bonyolódott zusc...
3       rendvédelmi szervek védelmi szolgálata rszvsz ...
4       b build trade kft építheti sportcsarnokot foci...
9       mvm nél belső vizsgálat zajlik, milliárd forin...
11      gyakorlatilag visszavonta feltáró jellegű nyom...
                              ...                        
3611    fideszben többen biztosra veszik, schmitt pál ...
3612    magyar kormány kezdeményezni fogja norvég alap...
3614    fatf financial action task force elnöke clarie...
3616    adócsalás megkezdődött olmert pere izraelben m...
3617    bkv zrt vállalat pályázatain nyertes cégek köz...
Name: clean_text, Length: 28381, dtype: object

In [44]:
df['clean_stemmed_text']

2       rendű vádlot szerelm kalan bonyolódot zuschlag...
3       rendvédelm szerv védelm szolgálat rszvsz , töb...
4       b buil trad kft építhet sportcsarn focipály pé...
9       mvm nél belső vizsgál zajlik, milliár forin ke...
11      gyakorlatilag visszavont feltáró jellegű nyomo...
                              ...                        
3611    fidesz töb biztos veszik, schmit pál lemondás ...
3612    magyar kormány kezdeményezn fog norvég alap ke...
3614    fatf financial act tas forc eln clar lo csütör...
3616    adócsalás megkezdődöt olmer pe izrael megkezdő...
3617    bkv zrt vállal pályázat nyertes cég közötti, h...
Name: clean_stemmed_text, Length: 28381, dtype: object

In [45]:
vectorizer = CountVectorizer(min_df = 0.005, max_df = 0.75)
# vectorizer = TfidfVectorizer(norm = None, sublinear_tf=True)
BoW_Matrix = vectorizer.fit_transform(df['clean_stemmed_text'])

In [46]:
BoW_df = pd.DataFrame(BoW_Matrix.toarray(), columns = vectorizer.vocabulary_)
BoW_df.head(3)

,vádlot,zuschlag,egyéb,állítja,volt,pályázat,aláír,nev,per,kez,...,államosítás,petőf,árfolyam,segély,bulgár,felmutatn,megszegés,fűződi,túlzot,adókedvezmény
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,1,0,0,0,0


In [47]:
stop_words = get_stop_words()
stemmed_stop_words = [stem_text(word) for word in stop_words]
len(stemmed_stop_words)

828

In [48]:
len(np.unique(stemmed_stop_words))

502

In [49]:
BoW_df=BoW_df[BoW_df.columns[~BoW_df.columns.isin(np.unique(stemmed_stop_words + stop_words))]]
BoW_df

,vádlot,zuschlag,állítja,pályázat,aláír,nev,per,kez,forduln,meghallgatás,...,államosítás,petőf,árfolyam,segély,bulgár,felmutatn,megszegés,fűződi,túlzot,adókedvezmény
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28376,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28377,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28378,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28379,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
pca = PCA(n_components=1500)
pca_bow = pca.fit_transform(BoW_df.to_numpy())
reducted_features = pd.DataFrame(pca_bow)
reducted_features.index = df.index
reducted_features

,0,1,2,3,4,5,6,7,8,9,...,1490,1491,1492,1493,1494,1495,1496,1497,1498,1499
2,-2.335878,1.207352,1.253296,3.200807,-0.208290,2.009210,-1.316157,1.325700,1.729711,0.730459,...,-0.137147,0.117512,-0.250312,-0.519946,0.042403,0.275957,0.018694,0.311641,-0.008211,-0.156678
3,-2.430420,1.950404,3.992161,2.830340,-0.818367,1.697134,-2.358949,2.862256,2.891890,-0.536769,...,0.363079,0.239774,0.186610,-0.088605,-0.258017,-0.104618,0.035401,-0.125492,-0.308523,0.195725
4,13.641859,-8.581301,-13.866707,-3.739752,-2.862452,-4.602806,-4.534167,-1.933218,2.260488,1.714220,...,0.233141,0.443982,-0.160243,-0.280394,-0.897312,0.248557,-0.072387,-0.013443,-0.415883,-0.542265
9,11.997657,-20.703459,-6.306257,7.911571,7.876504,9.816929,5.529835,8.562952,-10.894462,-0.814783,...,-0.014611,0.346329,0.219029,-0.146656,0.275724,-0.677509,-0.584724,0.609409,-0.244067,-0.197959
11,-1.245506,-0.705393,-0.034206,4.146588,0.082642,2.459017,-2.058165,2.975640,-2.537849,0.646931,...,-0.044016,-0.045718,0.007871,-0.301363,-0.338643,0.157045,-0.106199,-0.076708,-0.192229,0.101450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3611,-3.449542,3.227729,1.400784,-0.568819,-0.036166,-0.868196,2.491588,0.134793,0.935349,-0.930292,...,0.055092,0.052776,0.064207,-0.024522,-0.173246,0.049758,0.018420,-0.094171,-0.140706,0.087674
3612,-2.611382,0.415831,1.658573,-3.701168,1.330298,1.471474,2.042069,-0.746901,0.268736,2.346906,...,0.353595,0.075705,0.134511,-0.088537,-0.314953,0.035477,-0.383697,-0.183891,-0.016766,0.278325
3614,-3.599000,2.968867,1.190378,-1.683578,1.011069,0.711617,1.364993,-0.264555,0.038608,0.729370,...,0.039645,0.006129,-0.704340,-0.263065,0.101488,-0.236057,0.039800,-0.350911,-0.181202,0.130744
3616,-4.633083,4.748636,-0.959533,-0.393582,0.818055,0.073724,0.927543,0.143336,-0.103383,-0.157252,...,-0.016332,0.005883,-0.006100,0.031309,-0.122651,-0.081941,-0.115588,0.035515,-0.083167,-0.159759


# An example fit-predict

In [51]:
target_tag = tag_columns[0]
print(target_tag)
print('target tag freq ', df[target_tag].sum()/28381)
model = svm.SVC()
model = model.fit(reducted_features, df[target_tag])
model.score(reducted_features, df[target_tag])

tag_ingatlan
target tag freq  0.11345618547619886


0.9558155103766605

In [52]:
classifier = RandomForestClassifier(random_state = 2)
classifier=classifier.fit(reducted_features,df[target_tag])
accuracy_score(df[target_tag], classifier.predict(reducted_features))

0.99996476515979

# Looping through set of tags, fit-predict

In [53]:
def train_test_ratio_split(df, reducted_features, target_tag):
    train_x = reducted_features[:round(0.8*len(reducted_features))]
    train_y = df[target_tag][:round(0.8*len(reducted_features))]
    test_x = reducted_features[round(0.8*len(reducted_features)):]
    test_y = df[target_tag][round(0.8*len(reducted_features)):]
    return train_x, train_y, test_x, test_y

def train_test_ratio_split_reduce_negative_samples(df, reducted_features, target_tag, min_positive_ratio = 0.3):
    n_positive_samples = df[target_tag].sum()
    n_negative_samples = len(df) - n_positive_samples
    if n_positive_samples/ len(df) > min_positive_ratio:
        keep_samples = df.index
    else:
        n_negative_samples = round(n_positive_samples * 7 / 3)
        keep_samples =sorted(
                list(df[df[target_tag] == 1].index.values) + \
                list(random.sample(list(df[df[target_tag] == 0].index), n_negative_samples))
            )
    n = len(keep_samples)
    train_x = reducted_features.loc[keep_samples][:round(0.8*n)]
    train_y = df.loc[keep_samples, target_tag][:round(0.8*n)]
    test_x = reducted_features.loc[keep_samples][round(0.8*n):]
    test_y = df.loc[keep_samples, target_tag][round(0.8*n):]

    return train_x, train_y, test_x, test_y

In [54]:
target_tags = tag_stats.index[:10]

In [55]:
results = pd.DataFrame(index = target_tags, columns = ['TP', 'FP', 'FN', 'TN', 'test_freq', 'train_freq'])

In [62]:
for target_tag in target_tags:
    print(target_tag)
    # train_x, train_y, test_x, test_y = train_test_ratio_split(df, reducted_features, target_tag)
    train_x, train_y, test_x, test_y = train_test_ratio_split_reduce_negative_samples(df, reducted_features, target_tag, min_positive_ratio = 0.3)

    classifier = RandomForestClassifier(random_state = 2)
    classifier=classifier.fit(train_x,train_y)
    y_pred = classifier.predict(test_x)

    results.loc[target_tag, 'TP'] = ((test_y == 1) & (y_pred == 1)).sum()
    results.loc[target_tag, 'FP'] = ((test_y == 0) & (y_pred == 1)).sum()
    results.loc[target_tag, 'FN'] = ((test_y == 1) & (y_pred == 0)).sum()
    results.loc[target_tag, 'TN'] = ((test_y == 0) & (y_pred == 0)).sum()
    results.loc[target_tag, 'test_freq'] = round(test_y.sum()/len(test_y), 2)
    results.loc[target_tag, 'train_freq'] = round(train_y.sum()/len(train_y), 2)

    print(classification_report(test_y, y_pred))

    print('\tactivity ratio:', round(df[target_tag].sum()/len(df[target_tag]), 2))
    print('\taccuracy: ', accuracy_score(test_y, y_pred)) 
    print('\tTP: ', ((test_y == 1) & (y_pred == 1)).sum()) 
    print('\tFP: ', ((test_y == 0) & (y_pred == 1)).sum()) 
    print('\tFN: ', ((test_y == 1) & (y_pred == 0)).sum()) 
    print('\tTN: ', ((test_y == 0) & (y_pred == 0)).sum()) 

tag_önkormányzat
              precision    recall  f1-score   support

           0       0.73      0.98      0.83      3932
           1       0.77      0.16      0.27      1739

    accuracy                           0.73      5671
   macro avg       0.75      0.57      0.55      5671
weighted avg       0.74      0.73      0.66      5671

	activity ratio: 0.22
	accuracy:  0.7279139481572915
	TP:  282
	FP:  86
	FN:  1457
	TN:  3846
tag_klientúra
              precision    recall  f1-score   support

           0       0.74      0.97      0.84      2559
           1       0.72      0.20      0.32      1099

    accuracy                           0.74      3658
   macro avg       0.73      0.59      0.58      3658
weighted avg       0.73      0.74      0.68      3658

	activity ratio: 0.14
	accuracy:  0.7372881355932204
	TP:  225
	FP:  87
	FN:  874
	TN:  2472
tag_közbeszerzés
              precision    recall  f1-score   support

           0       0.77      0.98      0.86      2280
  

In [57]:
results['FalseDiscoveryRate'] = results['FP'] / (results['FP'] + results['TP'])
results

,TP,FP,FN,TN,test_freq,train_freq,FalseDiscoveryRate
tag,,,,,,,
tag_önkormányzat,295,93,1446,3848,0.31,0.29,0.239691
tag_klientúra,198,89,889,2444,0.3,0.29,0.310105
tag_közbeszerzés,289,42,661,2245,0.29,0.28,0.126888
tag_ingatlan,188,32,672,2051,0.29,0.29,0.145455
tag_támogatás,217,61,651,1989,0.3,0.28,0.219424
tag_átláthatóság,101,43,745,1971,0.3,0.28,0.298611
tag_csalás,143,14,696,1896,0.31,0.28,0.089172
tag_háttér,187,146,563,1648,0.29,0.28,0.438438
tag_gazdálkodás,41,30,616,1490,0.3,0.28,0.422535
